# Hackathon

Импортируем модули

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import catboost as ctb
import tensorflow as tf

Читаем файлы для трэйна и теста

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

Для начала разберемся какие типы данных у нас присутствуют 

In [3]:
df_train.dtypes

Year                   int64
Date                  object
Locality              object
Address               object
Estimated Value      float64
Sale Price           float64
Property              object
Residential           object
num_rooms              int64
carpet_area            int64
property_tax_rate    float64
dtype: object

У нас присутствуют категориальные признаки(Locality, Property, Residential), также есть дата, и множество числовых данных. Для начала разберемся с датой, год уже присутствует, остается только вытащить месяц и день, это потенциально может повлиять на цену, ну и затем уберем дату и адрес(они нам больше не понадобятся)

In [4]:
df_train['Date'] = pd.to_datetime(df_train['Date'])
df_train['month']= df_train['Date'].dt.month
df_train['day']= df_train['Date'].dt.day
df_test['Date'] = pd.to_datetime(df_test['Date'])
df_test['month']= df_test['Date'].dt.month
df_test['day']= df_test['Date'].dt.day

In [5]:
df_train.drop(['Date','Address'], axis=1, inplace=True)
df_test.drop(['Date','Address'], axis=1, inplace=True)

Дальше разберемся с категориальными признаками. Property и Residential ввиду малого кол-ва уникальных значений просто воспользуемся OneHotEncoder, а с Locality так не прокатит(там 170 уникальных значений), к нему применим обычный LabelEncoder

In [6]:
df_train[['Locality']].nunique(), df_train[['Property']].nunique(), df_train[['Residential']].nunique()

(Locality    170
 dtype: int64,
 Property    6
 dtype: int64,
 Residential    5
 dtype: int64)

In [7]:
# ohe = OneHotEncoder()
# transformed = ohe.fit_transform(df_ohe[['Locality']]) 
ohe1 = OneHotEncoder()
transformed1 = ohe1.fit_transform(df_train[['Property']]) 
ohe2 = OneHotEncoder()
transformed2 = ohe2.fit_transform(df_train[['Residential']]) 

In [8]:
# df_ohe[ohe.categories_[0]] = transformed.toarray()
df_train[ohe1.categories_[0]] = transformed1.toarray()
df_train[ohe2.categories_[0]] = transformed2.toarray()

In [9]:
lb = LabelEncoder()
df_train['Locality'] = lb.fit_transform(df_train['Locality'])

In [10]:
df_train.drop(['Property'], axis=1, inplace=True)

Осталось только разобраться с числовыми признаками, необходимо стандартизировать Sale price и Estimated Value, это улучшит работу модели и нам никак не помешает

In [11]:
scaler = StandardScaler()
df_train["Estimated Value"] = scaler.fit_transform(df_train["Estimated Value"].values.reshape(-1,1))
df_train["Sale Price"] = scaler.fit_transform(df_train["Sale Price"].values.reshape(-1,1))

Разобьем данные на train и validation выборки

In [12]:
x_train, x_test = train_test_split(df_train.drop('Sale Price', axis=1), train_size=0.7, random_state=1)
y_train, y_test = train_test_split(df_train['Sale Price'], train_size=0.7, random_state=1)

Я собираюсь построить множество различных моделей(Catboost, RandomForest, Linear) и нейронку для регрессии

In [13]:
model = RandomForestRegressor(n_jobs=-1, random_state=27)
# Для начала обучим модель без настройки гиперпараметров
model.fit(x_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=27)

In [14]:
pred_test = model.predict(x_test)
pred_train = model.predict(x_train)

In [15]:
r2_score(y_train, pred_train), mean_absolute_error(y_train, pred_train)

(0.954505187157205, 0.04335825243732958)

In [16]:
r2_score(y_test, pred_test), mean_absolute_error(y_test, pred_test)

(0.5216313825333798, 0.1157429670919535)

Всего 50% дисперсии на тесте и 96% на трейне, признаки переобучения, попробуем настроить max_depth

In [ ]:
tree = RandomForestRegressor(n_jobs=-1, random_state=27)

param_grid = {'max_depth': [5, 10, 15, 20]}
cv = 5

grid_search = GridSearchCV(tree, param_grid=param_grid, cv=cv)
grid_search.fit(x_train, y_train)

print(grid_search.best_params_)

Лучше всего себя показал результат с max_depth 15, дальше настроим min_samples_leaf

In [ ]:
tree = RandomForestRegressor(max_depth=15, n_jobs=-1, random_state=27, verbose=10)

param_grid = {'min_samples_leaf': [1, 5, 10, 15, 20]}
cv = 5

grid_search = GridSearchCV(tree, param_grid=param_grid, cv=cv)
grid_search.fit(x_train, y_train)

print(grid_search.best_params_)

Лучше всего оставить min_samples_leaf без изменения

In [18]:
model = RandomForestRegressor(max_depth=15, n_jobs=-1, random_state=27)

model.fit(x_train, y_train)

RandomForestRegressor(max_depth=15, n_jobs=-1, random_state=27)

In [19]:
pred_test = model.predict(x_test)
pred_train = model.predict(x_train)

In [20]:
r2_score(y_train, pred_train), mean_absolute_error(y_train, pred_train)

(0.9108009675526895, 0.0955702211590535)

In [21]:
r2_score(y_test, pred_test), mean_absolute_error(y_test, pred_test)

(0.53192669128142, 0.11563419589839538)

С настроенными гиперпараметрами результат улучшился на 4 процента(

Пусть это будет первая модель для отправки

In [23]:
df_test['Locality'] = lb.transform(df_test['Locality'])
transformed_t = ohe1.transform(df_test[['Property']]) 
transformed_t1 = ohe2.transform(df_test[['Residential']]) 
df_test[ohe1.categories_[0]] = transformed_t.toarray()
df_test[ohe2.categories_[0]] = transformed_t1.toarray()
df_test.drop('Property', axis=1, inplace=True)

In [24]:
df_test['Estimated Value'] = scaler.transform(df_test["Estimated Value"].values.reshape(-1,1))

In [25]:
df_test_for_model = df_test.drop(['Segment', 'Sale Price'], axis=1)

In [26]:
pred = model.predict(df_test_for_model)

df_test['Sale Price'] = pred

df_test['Profit'] = (df_test['Sale Price'] - df_test['Estimated Value'])/100

Осталось только разбить данные на сегменты, чтобы стандартизация нам никак не помешала, разобьем данные с помощью квантилей

In [27]:
q1, q2, q3 = df_test['Profit'].quantile([0.25, 0.5, 0.75])
df_test['Segment'] = np.where(df_test['Profit'] >= q3, 0, np.where(df_test['Profit'] >= q2, 1, np.where(df_test['Profit'] >= q1, 2, 3)))

In [320]:
df_sub = pd.read_csv('submission.csv')

df_sub['Segment'] = df_test['Segment'] 

df_sub.to_csv('my_submission_file.csv', index=False)

Данный способ позволяет предсказать около 26% сегментов, что на самом деле очень плохо попробуем Catboost model 

В catboost изначально стоят неплохо настроенные гиперпараметры, так что сразу обучаем

In [29]:
model = ctb.CatBoostRegressor(random_state=27, verbose=0)

model.fit(x_train, y_train)

In [30]:
pred_train = model.predict(x_train)
pred_test = model.predict(x_test)

In [31]:
print(r2_score(y_train, pred_train), mean_absolute_error(y_train, pred_train))

0.8359267477858133 0.11364517014080265


In [32]:
print(r2_score(y_test, pred_test), mean_absolute_error(y_test, pred_test))

0.517034593123481 0.1210294094835803


51% дисперсии при обучении, но если отправить при остальных тех же условиях accuracy достигает 30%

Линейная модель умудряется описать только 10% дисперсии(это очень мало), так что перейду сразу к нейронкам

Долго настраивая слои, активации и rate в дропоуте у меня получилась такая нейронка

In [329]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, input_shape=x_train.iloc[0].shape, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.01),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(rate=0.05),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dropout(rate=0.05),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])


In [330]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.mean_absolute_error,
              metrics=[tf.keras.metrics.mean_squared_error,
                       tf.keras.metrics.mean_absolute_percentage_error
                       ])

In [331]:
def train_val_test_split(x, val_frac=0.15, test_frac=0.15):
    x_train = x[:round((1 - val_frac - test_frac) * len(x))]
    x_val = x[round((1 - val_frac - test_frac) * len(x)):round((1 - test_frac) * len(x))]
    x_test = x[round((1 - test_frac) * len(x)):]
    return x_train, x_val, x_test


x_train, x_val, x_test = train_val_test_split(df_ohe_w.drop('Sale Price', axis=1))
y_train, y_val, y_test = train_val_test_split(df_ohe_w['Sale Price'])

In [332]:
annealing = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                 factor=0.1,
                                                 patience=5,
                                                 verbose=1,
                                                 min_delta=0.001)

In [333]:
tb_callback = tf.keras.callbacks.TensorBoard(log_dir='logs/first', 
                                             histogram_freq=1)

In [262]:
model.fit(x_train, y_train, 
          validation_data=(x_val, y_val),
          batch_size = 1000,
          callbacks=[tf.keras.callbacks.EarlyStopping(patience=20), annealing, tb_callback],
          epochs=100)

Epoch 1/100
388/388 [==============================] - 5s 14ms/step - loss: 0.1693 - mean_squared_error: 0.4839 - mean_absolute_percentage_error: 197.1369 - val_loss: 0.1395 - val_mean_squared_error: 1.7502 - val_mean_absolute_percentage_error: 162.4903 - lr: 1.0000e-14
Epoch 2/100
388/388 [==============================] - 5s 13ms/step - loss: 0.1685 - mean_squared_error: 0.4915 - mean_absolute_percentage_error: 188.9921 - val_loss: 0.1393 - val_mean_squared_error: 2.1507 - val_mean_absolute_percentage_error: 164.3531 - lr: 1.0000e-14
Epoch 3/100
388/388 [==============================] - 5s 14ms/step - loss: 0.1684 - mean_squared_error: 0.4849 - mean_absolute_percentage_error: 193.8125 - val_loss: 0.1397 - val_mean_squared_error: 1.8431 - val_mean_absolute_percentage_error: 164.9094 - lr: 1.0000e-14
Epoch 4/100
388/388 [==============================] - 6s 15ms/step - loss: 0.1682 - mean_squared_error: 0.4877 - mean_absolute_percentage_error: 193.1836 - val_loss: 0.1402 - val_mean_sq

In [259]:
pred = model.predict(x_test)

2597/2597 [==============================] - 4s 2ms/step


In [260]:
r2_score(y_test, pred)

0.31731232675449605

Нейронка с трудом описывает 30% дисперсии(не подходит), на последок решил себя проверить использовав встроенную нейронку из sklearn, она описала лишь 27% дисперсии(все мимо)

In [62]:
from sklearn.neural_network import MLPRegressor

In [ ]:
model = MLPRegressor(learning_rate='adaptive', random_state=27, max_iter=500, verbose=1, n_iter_no_change=50)

model.fit(x_train, y_train)

In [65]:
pred = model.predict(x_test)

In [66]:
r2_score(y_test, pred)

0.2756038968572416

## Results

In [337]:
pd.DataFrame([[0.09673426871284, '<5%'],[0.31731232675449605,'15%'], [0.2756038968572416, '15%'],[0.5414108541853, '26%-27%'],[0.517034593123481, '30%']], columns=['R2_score', 'Accuracy_on_segm'], index=['Linear', 'DeepLearn', 'MLPReg', 'RandomForest', 'CatBoost'])

,R2_score,Accuracy_on_segm
Linear,0.096734,<5%
DeepLearn,0.317312,15%
MLPReg,0.275604,15%
RandomForest,0.541411,26%-27%
CatBoost,0.517035,30%
